In [23]:
#The purpose of this notebook is to compute the basin stability of a network governed by the coupled oscillator model
#Import necessary packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.integrate
import scipy.stats
from scipy.linalg import circulant
from scipy.optimize import curve_fit
import time
import networkx as nx



In [309]:
########################
### DEFINE FUNCTIONS ###
########################

#Generate all phase and frequency initial conditions we want to test 
def create_ensembles(T, N):
    # Define synchronous phase and frequency states
    θ_s = np.arcsin(1)   #synchronous phase state is arcsin(P/K) according to the Kurths paper
    ω_s = 0  # synchronous frequency

    # Define parameters for generating initial conditions (create Q)
    theta_min = 0
    theta_max = 2 * np.pi
    omega_min = -100
    omega_max = 100

    # Generate T initial conditions for phase and frequency
    theta_values = np.random.uniform(low=theta_min, high=theta_max, size=T)
    omega_values = np.zeros(T) #np.random.uniform(low=omega_min, high=omega_max, size=T)
    
    # Create initial entries for phases and frequencies
    initial_values = list(zip(theta_values, omega_values))
    first_entries = [val[0] for val in initial_values]
    second_entries = [val[1] for val in initial_values]
    
    # Initialize the ensembles for phases and frequencies
    phase_ensembles = []
    freq_ensembles = []
    
    # Create phase ensembles
    for i in range(N):
        phase_list = []
        for entry in first_entries:
            phase = [θ_s] * N  # All values set to synchronous state
            phase[i] = entry  # Set one specific position to entry value
            phase_list.append(tuple(phase))
        phase_ensembles.append(phase_list)  # Append the inner list to the outer list
    
    # Create frequency ensembles
    for i in range(N):
        freq_list = []
        for entry in second_entries:
            freq = [ω_s] * N  # All values set to synchronous state
            freq[i] = entry  # Set one specific position to entry value
            freq_list.append(tuple(freq))
        freq_ensembles.append(freq_list)  # Append the inner list to the outer list


    return phase_ensembles, freq_ensembles


#Our model
def kuramoto_function(phase,t,K,N,omega):
    phase_matr = np.tile(phase,(N,1))
    temp = np.multiply(K,np.sin(phase_matr-phase_matr.T))
    incr = omega + np.sum(temp,axis=1)
    return (incr)


#Simulating our model
def simulate_kuramoto(kuramoto, initial_θ, initial_ω, t):
    sol =scipy.integrate.odeint(kuramoto,y0=initial_θ,t=t,args=(K_ij,N,initial_ω),hmax=0.1,full_output=0,rtol=1e-4,atol=1e-4)
    last_value= sol[-1]
    ode_sol = kuramoto_function(sol[-1],0,K_ij,N,initial_ω)
    return sol, np.round(last_value, 6), np.round(ode_sol, 6)

#Computing the basin stability for each node
def compute_basin_stability(results, T):


    #COMPUTES U VALUES (i.e. the number of times phase/ freq synchronizes)
    # Determine the maximum outer_index to initialize counts
    node_max = max(res['outer_index'] for res in results)

    # Initialize counts for `last_phase` and `ode_value` by `outer_index`
    U_phase = [0] * (node_max + 1)
    U_freq = [0] * (node_max + 1)

    # Check for identical `last_phase` and `ode_value` values by node
    for outer_index in range(node_max + 1):
        # Get all `last_phase` and `ode_value` for this node
        last_phases = [res['last_phase'] for res in results if res['outer_index'] == outer_index]
        ode_values = [res['ode_value'] for res in results if res['outer_index'] == outer_index]

        # If all `last_phase` arrays are the same, increment the count
        if all(np.array_equal(last_phases[0], lp) for lp in last_phases):
            U_phase[outer_index] += 1

        # If all `ode_value` arrays are the same, increment the count
        if all(np.array_equal(ode_values[0], ov) for ov in ode_values):
            U_freq[outer_index] += 1

    
    #COMPUTES BASIN STABILITY PER NODE
    phase_basin=  [x / T for x in U_phase]
    freq_basin = [x / T for x in U_freq]
    

    return phase_basin, freq_basin

    


In [310]:

##########################
#### MODEL PARAMETERS #### 
##########################

T= 2 # number of test ensembles
N= 3 #number of nodes


#Time iterations for ODE solver
t_max = 10000
t_min = 0
t = np.arange(t_min,t_max,1)

#Create Network
G=nx.complete_graph(N)
K_ij=nx.to_numpy_array(G)
#nx.draw(G)

In [311]:

##########################
#### SIMULATION #### 
##########################

# Initialize the list to store results
results = []

# Iterate through every pair of (phase_ensembles, freq_ensembles)
for outer_index in range(len(phase_ensembles)):
    for inner_index in range(len(phase_ensembles[outer_index])):
        initial_phase = phase_ensembles[outer_index][inner_index]
        initial_freq = freq_ensembles[outer_index][inner_index]

        # Simulate the Kuramoto system with the given initial conditions
        simulation_result = simulate_kuramoto(
            kuramoto_function,
            initial_θ=initial_phase,
            initial_ω=initial_freq,
            t=t
        )

        # Store the result
        results.append({
            'outer_index': outer_index,
            'inner_index': inner_index,
            'simulation': simulation_result[0],  # The solution from the ODE solver
            'last_phase': simulation_result[1],  # The last phase value
            'ode_value': simulation_result[2]  # The output from kuramoto_function
        })


In [312]:
##########################
######## RESULTS #########
##########################
for res in results:
    print(f"Outer Index: {res['outer_index']}, Inner Index: {res['inner_index']}")
    print("Last State Phase Value:", res['last_phase'])
    print("ODE Value:", res['ode_value'])
    print("----")


Outer Index: 0, Inner Index: 0
Last State Phase Value: [236907.759876    526.050162    526.050162]
ODE Value: [21.975324  0.911255  0.911255]
----
Outer Index: 0, Inner Index: 1
Last State Phase Value: [-9.68107527e+05 -8.35975630e+01 -8.35975630e+01]
ODE Value: [-98.763563   0.962927   0.962927]
----
Outer Index: 1, Inner Index: 0
Last State Phase Value: [   522.401404 236915.057392    522.401404]
ODE Value: [ 0.366436 23.06496   0.366436]
----
Outer Index: 1, Inner Index: 1
Last State Phase Value: [-8.7201308e+01 -9.6810032e+05 -8.7201308e+01]
ODE Value: [ -0.441694 -95.954322  -0.441694]
----
Outer Index: 2, Inner Index: 0
Last State Phase Value: [   473.863677    473.863677 237012.132846]
ODE Value: [ 0.995424  0.995424 21.806985]
----
Outer Index: 2, Inner Index: 1
Last State Phase Value: [-8.69422330e+01 -8.69422330e+01 -9.68100838e+05]
ODE Value: [ -0.944052  -0.944052 -94.949604]
----


In [313]:
compute_basin_stability(results, T)

([0.0, 0.0, 0.0], [0.0, 0.0, 0.0])